A notebook to test data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
import pandas as pd
from natsort import natsorted

from scripts.create_problem import load_problem_set
from scripts.run_solver import load_results
from notebooks.utils import *

Checking if a dataset (train, validation or test) contains separable problem classes. It means that if the dataset consists of $n$ different QUBO problem types, for example, Knapsack, MaxCut, etc, the corresponding martixes Q make spatially separable groups in the $\mathbb{R}^{d \times d}$ space according to their problem type. 

In [ ]:
# read_dir = '../data/test'
# read_dir = "../data/normal_10"
# read_dir = "../data/various"
read_dir = "../data/normal"
# read_dir = "../data/distributions"
problem_list = natsorted(os.listdir(read_dir))

# problem_list = [
#     'SPP',
#     'GraphColoring',
#     'Knapsack',
#     'MVC',
#     'QUBO',
#     'SetPack',
#     'WMaxCut',
#     'NumberPartitioning',
#     'MaxCut',
#     'WMVC',
#     'Ising',
#     'Max2Sat',
#     'QAP'
# ]

X, y = get_Xy(read_dir, problem_list, suffix='test')
X_tsne = get_tsne(X)
show_tsne(problem_list, X_tsne, y)
print_unique(read_dir, problem_list)

Get the optimization trajectoriy for every problem from the dataset obtained with different solvers and average them over a problem type.

In [ ]:
# read_dir = '../results/test'
# read_dir = '../results/normal_10'
# read_dir = '../results/various'
read_dir = '../results/normal'
# read_dir = '../results/distributions'

problem_list = natsorted(os.listdir(read_dir))
solver_list = ('RandomSearch', 'PSO', 'PROTES')

meta_results = defaultdict(dict)
meta_stats = defaultdict(dict)
for problem in problem_list:
    for solver in solver_list:
        meta_results[problem][solver] = get_meta_results(problem, solver, read_dir, suffix='test', budget=5000, n_steps=100)
        meta_stats[problem][solver] = get_trajectory_stats(problem, solver, read_dir, suffix='test')

Here one can go to `utils.py` file and adjust the printing settings. Namely, it is possible to turn off clipping or add information about std.

In [ ]:
show_meta_results(meta_results)

To get one trajectory for a convinient printing average over the problem types as well. Also the trajectories statistics are accumulated as well.

In [ ]:
title = 'Normal'
problem_averaged_meta_results = get_problem_averaged_meta_dict(meta_results, title)
problem_averaged_meta_stats = get_problem_averaged_meta_dict(meta_stats, title)

show_meta_results(problem_averaged_meta_results)
pd.DataFrame(problem_averaged_meta_stats[title]).T.round(2)

Stuff

In [ ]:
# flattened = {
#     (row_key, col_key): subdict
#     for row_key, cols in meta_stats.items()
#     for col_key, subdict in cols.items()
# }
# pd.DataFrame.from_dict(flattened, orient='index')

In [ ]:
# from scripts.create_problem import load_problem_set
# from scripts.run_solver import load_results

# d_x = []
# problem_name = 'QAP'
# problems = load_problem_set(f'{read_dir}/{problem_name}/train')
# for problem in problems:
#     d_x.append(problem.info['x_best'])
#     print(problem.Q)
# x_unique = torch.unique(torch.stack(d_x), dim=0)
# print(x_unique)